In [49]:
 # Get a list of keys with 'null' values
null_keys = [key for key, value in sample_data_updated.items() if value == 'null']

# Limit to 5 questions or less
questions_to_ask = null_keys[:3]

    # what to put into model to ask next question
transcript = (
f"""
{questions_to_ask}
"""
    )

stream = client.chat.completions.create(
model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
messages=[
            {
                "role": "system",
                "content": """You are an assistant helping someone set up a buisness.
                You are to use the missing content to ask the client for more information.""",
            },

            {
                "role": "user",
                "content": transcript,
            }

        ]
)

stream.choices[0].message.content
#stream['ChatCompletionResponse'] #['message']['content']

'To help you set up your business, I need a bit more information. \n\nCould you please provide me with the following details:\n\n1. What is the name of the city where your business is located?\n2. What is the zip code of your business location?\n3. What is the email address you would like to use for your business?'

In [2]:
from dotenv import load_dotenv
import os
from together import Together

load_dotenv()


client = Together(api_key=os.getenv("TOGETHER_API_KEY"))


sample_data_all_null = {
        '(Name of Business DBA)': 'null',
        '(Business Phone)': 'null',
        '(Business Address include street directions and suite number if applicable)': 'null',
        '(City)': 'null',
        '(Zip)': 'null',
        '(Business EMail)': 'null',
        '(Seating  Bed Capacity Licensed Healthcare)': 'null',
        '(Square Footage)': 'null',
        '(hours)': 'null',
        '(LP)': 'null',
        '(LLP)': 'null',
        '(Corporation)': 'null',
        '(LLC)': 'null',
        '(Business Owner)': 'null',
        '(Owner Phone)': 'null'
    }

# you start off by not knowing anything
print("hello! do you want to tell me a little about the buissness you're trying to set up?")

#####
# you're code
####

first_input = ""

sample_data_updated = sample_data_all_null 

# extracted from the input text 
update = {'(Name of Business DBA)': 'cookies inc.', '(Business Phone)': '860-941-0195'}

# Update the sample_data_updated dictionary with the values from the update dictionary
sample_data_updated.update(update)



hello! do you want to tell me a little about the buissness you're trying to set up?


In [14]:
#### json extractor

import json
from together import Together
from pydantic import BaseModel, Field
import together

together = Together(api_key=os.getenv("TOGETHER_API_KEY"))

# Define the schema for the output
class flexible_outputs(BaseModel):
    """Only extract the core info, don't give any additional info"""

    Name_of_Business: str = Field(description="The name of the business")


    Business_Phone: str = Field(description="The phone number of the business")


    Business_Address: str = Field(description="The address of the business")
    
  
#City: str = Field(description="The city of the business if not given put null")
#zip: str = Field(description="The zipcode of the business if not given put null")
#Business_email: str = Field(description="The email of the business if not given put null")
#seat_capacity: str = Field(description="The seating capacity of the business if not given put null")
#Square_Footage: str = Field(description="The square footage the business if not given put null")
#hours: str = Field(description="The hours of operation of the business if not given put null")
#Business_owner: str = Field(description="The users first and/or last name if not given put null")
#owner_phone_number: str = Field(description="The users personal phone number if not given put null")

def main():
    transcript = (
        "Good morning! so it's called cookies inc. it's on 24th street and the number is 860-941-0195"
    )

    # Call the LLM with the JSON schema
    extract = together.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "The following is a voice message transcript. Only answer in JSON.",
            },
            {
                "role": "user",
                "content": transcript,
            },
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": flexible_outputs.model_json_schema(),
        },
    )

    output = json.loads(extract.choices[0].message.content)
    # print(json.dumps(output, indent=2))
    return output

main()

{'Name_of_Business': 'Cookies Inc',
 'Business_Phone': '860-941-0195',
 'Business_Address': '24th street'}

# Function fin version

In [84]:
from dotenv import load_dotenv
import os
from together import Together

load_dotenv()


client = Together(api_key=os.getenv("TOGETHER_API_KEY"))


sample_data_all_null = {
        '(Name of Business DBA)': 'null',
        '(Business Phone)': 'null',
        '(Business Address include street directions and suite number if applicable)': 'null',
        '(City)': 'null',
        '(Zip)': 'null',
        '(Business EMail)': 'null',
        '(Seating  Bed Capacity Licensed Healthcare)': 'null',
        '(Square Footage)': 'null',
        '(hours)': 'null',
        '(LP)': 'null',
        '(LLP)': 'null',
        '(Corporation)': 'null',
        '(LLC)': 'null',
        '(Business Owner)': 'null',
        '(Owner Phone)': 'null'
    }

# you start off by not knowing anything
print("hello! do you want to tell me a little about the buissness you're trying to set up?")

#####
# you're code
####


dictionary_to_rename = {'Name_of_Business':'(Name of Business DBA)',
                        'Business_Phone':'(Business Phone)',
                        'Business_Address':'(Business Address include street directions and suite number if applicable)',
                        'persons_name':'(Business Owner)'}


sample_data_updated = sample_data_all_null


hello! do you want to tell me a little about the buissness you're trying to set up?


In [135]:


def ask_follow_up(sample_data_updated):

    # Get a list of keys with 'null' values
    null_keys = [key for key, value in sample_data_updated.items() if value == 'null'or value == 'N/A' or value == '']

    # Limit to 5 questions or less
    questions_to_ask = null_keys[:3]

     # what to put into model to ask next question
    transcript = (
    f"""
    {questions_to_ask}
    """
        )
    
    stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=[
                {
                    "role": "system",
                    "content": """You are an assistant helping someone set up a buisness.
                You are to use the list provided to task questions only related to that list""",
                },

                {
                    "role": "user",
                    "content": transcript,
                }

            ]
    )
    #print(stream.choices[0].message.content)
    
    return questions_to_ask, stream.choices[0].message.content

    for chunk in stream:
        return chunk.choices[0].delta.content



#### Function to find out if done or not and to update our final dictionary
def update_current_status(sample_data_updated, output, dictionary_to_rename):
    # input: current status dict
    # input newly gathered dict results
    # input: renaming dict
    # output: new dict

    # Programmatically rename the keys
    output = {
        dictionary_to_rename.get(key, key): value
        for key, value in output.items()
    }

    # Update the sample_data_updated dictionary with the values from the update dictionary
    sample_data_updated.update(output)

    print(sample_data_updated)

    # Count the number of fields still set to 'null'
    still_null_count = sum(1 for value in sample_data_updated.values() if value == 'null' or value == 'N/A' or value == '')

    # Calculate the percentage of the form that is complete
    total_fields = len(sample_data_updated)
    perc_complete = ((total_fields - still_null_count) / total_fields) * 100

    print(f'Number of fields still null: {still_null_count}, Percent of form complete: {perc_complete:.2f}%')

    if perc_complete == 100:
        print("done! no more chat")
        return sample_data_updated, still_null_count
    else:
        return sample_data_updated, still_null_count


# Define the schema for the output
class intial_response(BaseModel):
    """Only extract the core info, don't give any additional info"""

    Name_of_Business: str = Field(description="The name of the business")
    Business_Phone: str = Field(description="The phone number of the business")
    Business_Address: str = Field(description="The address of the business")

class persons_name(BaseModel):
    """Only extract the core info, don't give any additional info"""
    persons_name: str = Field(description="The name of the client")

class city(BaseModel):
    """Only extract the core info, don't give any additional info"""
    city_name: str = Field(description="The name of the city")

class zip(BaseModel):
    """Only extract the core info, don't give any additional info"""
    zipcode: str = Field(description="The zipcode of the buissness")

class email(BaseModel):
    """Only extract the core info, don't give any additional info"""
    email: str = Field(description="The email of the buissness")


def extract_values_from_response(response_gathering, transcript):
    
    output = {}
    for i in response_gathering:
        # select classes based in list_of_latest_questions_asked
        # Call the LLM with the JSON schema
        extract = together.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "you are a helpful assitant trying to take contxt out of a statement. Only answer in JSON.",
                },
                {
                    "role": "user",
                    "content": transcript,
                },
            ],
            model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
            response_format={
                "type": "json_object",
                "schema": i.model_json_schema(),
            },
        )

        output.update(json.loads(extract.choices[0].message.content))
        # print(json.dumps(output, indent=2))
        print(output)

    return output

    
    
list_of_latest_questions_asked = ''





In [136]:
dictionary_to_rename_classes = {'(Business Owner)':persons_name,
                                '(City)': city,
                                '(Zip)': zip,
                                '(Business EMail)': email,
                        }


In [137]:
# intially someone is going to tell something

transcript = """Hi my buisness name is midnight cookies the phone is 860-941-0195 and the address is 24th street"""

response_gathering = [intial_response]

output = extract_values_from_response(response_gathering, transcript)
print(output)

#### Function to find out if done or not and to update our final dictionary
sample_data_updated, num_still_null = update_current_status(sample_data_updated, output, dictionary_to_rename)

if num_still_null >= 11:
    questions_to_ask, question_script = ask_follow_up(sample_data_updated)

    print(questions_to_ask)

    print(question_script)

   # response_gathering = []
    response_gathering = [dictionary_to_rename_classes.get(field, field) for field in questions_to_ask]

    transcript = "okay i live in baltamore and my zipcode is 92342"

    print(response_gathering)
    output = extract_values_from_response(response_gathering, transcript)

    output

else:
    print('done')
    # convert that to class suggestion


{'Name_of_Business': 'Midnight Cookies', 'Business_Phone': '860-941-0195', 'Business_Address': '24th Street'}
{'Name_of_Business': 'Midnight Cookies', 'Business_Phone': '860-941-0195', 'Business_Address': '24th Street'}
{'(Name of Business DBA)': 'Midnight Cookies', '(Business Phone)': '860-941-0195', '(Business Address include street directions and suite number if applicable)': '24th Street', '(City)': 'null', '(Zip)': 'null', '(Business EMail)': 'null', '(Seating  Bed Capacity Licensed Healthcare)': 'null', '(Square Footage)': 'null', '(hours)': 'null', '(LP)': 'null', '(LLP)': 'null', '(Corporation)': 'null', '(LLC)': 'null', '(Business Owner)': 'null', '(Owner Phone)': 'null'}
Number of fields still null: 12, Percent of form complete: 20.00%
['(City)', '(Zip)', '(Business EMail)']
To help you set up your business, I'll need some information. Please provide the following:

1. What is the name of the city where your business is located?
2. What is the zip code of your business locati

In [120]:
print(output)

{'persons_name': 'Unknown'}


In [131]:
for i in response_gathering:
    print(i)

<class '__main__.city'>
<class '__main__.zip'>
<class '__main__.email'>


In [141]:
extract = together.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are a helpful assitant trying to take contxt out of a statement. only extract what they say dont debate it. Only answer in JSON.",
            },
            {
                "role": "user",
                "content": transcript,
            },
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": zip.model_json_schema(),
        },
    )

extract.choices[0].message.content

'{"zipcode": "92342"}'

In [134]:
output = {}
for i in response_gathering:
    # select classes based in list_of_latest_questions_asked
    # Call the LLM with the JSON schema
    extract = together.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are a helpful assitant trying to take contxt out of a statement. Only answer in JSON.",
            },
            {
                "role": "user",
                "content": transcript,
            },
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": city.model_json_schema(),
        },
    )

    output = extract.choices[0].message.content
    # print(json.dumps(output, indent=2))
    print(output)



{"persons_name": "unknown (you did not provide your name)" }
{"persons_name": "unknown"}
{"persons_name": "Unknown"}


In [91]:
response_gathering = [persons_name, city]

print(len(response_gathering))

2


In [ ]:
# so i'm gonna need everything as functions

# function 1: done or not?
    # if not how many left and go onto function 2

# function 2: ask question
    # 

# Function 3: extract dict values
    #

# then back to function 1 
    # 

# end of working functions version

In [66]:
# each feature will be it's on json... that's the best option
# Define the schema for the output

# Define the schema for the output
class intial_response(BaseModel):
    """Only extract the core info, don't give any additional info"""

    Name_of_Business: str = Field(description="The name of the business")
    Business_Phone: str = Field(description="The phone number of the business")
    Business_Address: str = Field(description="The address of the business")

class persons_name(BaseModel):
    """Only extract the core info, don't give any additional info"""
    persons_name: str = Field(description="The name of the client")

def main(response_gathering, transcript):
    # Call the LLM with the JSON schema
    extract = together.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are a helpful assitant trying to take contxt out of a statement. Only answer in JSON.",
            },
            {
                "role": "user",
                "content": transcript,
            },
        ],
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": response_gathering.model_json_schema(),
        },
    )

    output = json.loads(extract.choices[0].message.content)
    # print(json.dumps(output, indent=2))
    return output


output = main(response_gathering=intial_response, 
     transcript="Good morning! im on 24th street the buissness is called choclate empire and my number is 860-941-0195")

sample_data_updated = sample_data_all_null 

dictionary_to_rename = {'Name_of_Business':'(Name of Business DBA)',
                        'Business_Phone':'(Business Phone)',
                        'Business_Address':'(Business Address include street directions and suite number if applicable)',
                        'persons_name':'(Business Owner)'}

# Programmatically rename the keys
output = {
    dictionary_to_rename.get(key, key): value
    for key, value in output.items()
}

# Update the sample_data_updated dictionary with the values from the update dictionary
sample_data_updated.update(output)

print(sample_data_updated)

# Count the number of fields still set to 'null'
still_null_count = sum(1 for value in sample_data_updated.values() if value == 'null')

# Calculate the percentage of the form that is complete
total_fields = len(sample_data_updated)
perc_complete = ((total_fields - still_null_count) / total_fields) * 100

print(f'Number of fields still null: {still_null_count}, Percent of form complete: {perc_complete:.2f}%')

{'(Name of Business DBA)': 'Chocolate Empire', '(Business Phone)': '860-941-0195', '(Business Address include street directions and suite number if applicable)': '24th Street', '(City)': 'null', '(Zip)': 'null', '(Business EMail)': 'null', '(Seating  Bed Capacity Licensed Healthcare)': 'null', '(Square Footage)': 'null', '(hours)': 'null', '(LP)': 'null', '(LLP)': 'null', '(Corporation)': 'null', '(LLC)': 'null', '(Business Owner)': 'null', '(Owner Phone)': 'null'}
Number of fields still null: 12, Percent of form complete: 20.00%


In [67]:


# ask for a maximum of 5 questions at a time:
if still_null_count > 0: # this should be while 

    # choose 5 keys that are still null and print them as a list:
    # Get a list of keys with 'null' values
    null_keys = [key for key, value in sample_data_updated.items() if value == 'null']

    # Limit to 5 questions or less
    questions_to_ask = null_keys[:3]

    # print(questions_to_ask)

    # what to put into model to ask next question
    transcript = (
    f"""
    {questions_to_ask}
    """
        )
    
    stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=[
                {
                    "role": "system",
                    "content": """You are an assistant helping someone set up a buisness.
                    You are to use the missing content to ask the client for more information.""",
                },

                {
                    "role": "user",
                    "content": transcript,
                }

            ],
    stream=True
    )

    for chunk in stream:
        print(chunk.choices[0].delta.content or "", end="", flush=True)

To help you set up your business, I need a bit more information. Could you please provide the following details:

1. What is the name of your business?
2. What type of business is it (e.g. retail, service-based, food establishment, etc.)?
3. What is the full address of your business, including the city and zip code?
4. What is your business email address? (I see you've provided a placeholder, but I'd like to confirm the actual email address you'd like to use for your business communications.)
5. What services or products will your business offer?
6. Do you have a specific business structure in mind (e.g. sole proprietorship, partnership, LLC, corporation)?
7. Do you have a business name already registered, or would you like to register a new one?
8. Do you have a business phone number and physical location (e.g. office, storefront, etc.)?

Please provide these details, and I'll be happy to assist you further.

In [61]:
questions_to_ask, output =  ask_follow_up(sample_data_updated)

In [62]:
questions_to_ask

['(City)', '(Zip)', '(Business EMail)']

In [63]:
output 

"To better assist you in setting up your business, I need a bit more information. Could you please provide the following details:\n\n1. What is the name of your business?\n2. What type of business are you planning to start (e.g., retail, service-based, food establishment, etc.)?\n3. What is the name of the city where your business will be located?\n4. What is the zip code of your business location?\n5. What is your business email address? (I see you've provided a placeholder, but I'd like to confirm the actual email address you'd like to use for your business communications.)\n6. What is the nature of your business (e.g., sole proprietorship, partnership, LLC, corporation, etc.)?\n7. Do you have a business name registered with the state or local government?\n8. Do you have a business license or permit required by the city or state?\n9. What is your business's mission statement or purpose?\n10. Do you have a rough estimate of your startup costs and projected revenue?\n\nOnce I have this

In [ ]:
ask_follow_up(sample_data_updated)
update_current_status(output, dictionary_to_rename)

In [ ]:

# Count the number of fields still set to 'null'
still_null_count = sum(1 for value in sample_data_updated.values() if value == 'null')

# Calculate the percentage of the form that is complete
total_fields = len(sample_data_updated)
perc_complete = ((total_fields - still_null_count) / total_fields) * 100

print(f'Number of fields still null: {still_null_count}, Percent of form complete: {perc_complete:.2f}%')

# ask for a maximum of 5 questions at a time:
if still_null_count > 0: # this should be while 

    # choose 5 keys that are still null and print them as a list:
    # Get a list of keys with 'null' values
    null_keys = [key for key, value in sample_data_updated.items() if value == 'null']

    # Limit to 5 questions or less
    questions_to_ask = null_keys[:5]

    # print(questions_to_ask)

    # what to put into model to ask next question
    transcript = (
    f"""
    {questions_to_ask}
    """


        )
    
    stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    messages=[
                {
                    "role": "system",
                    "content": """You are an assistant helping someone set up a buisness.
                    You are to use the missing content to ask the client for more information.""",
                },

                {
                    "role": "user",
                    "content": transcript,
                }

            ],
    stream=True
    )

    for chunk in stream:
        print(chunk.choices[0].delta.content or "", end="", flush=True)